In [23]:
# Porcesaría las imagenes y las cargaría en el sistema
import cv2
import os

input_folder = r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset\train\no_fire"
output_folder = r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset\no_fire_procesado"

# Crea la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Carga la imagen
    img = cv2.imread(image_path)
    
    # Verifica si la imagen se cargó correctamente
    if img is None:
        print(f"Error al cargar la imagen: {image_path}")
        continue
    
    # Cambia tamaño
    resized_img = cv2.resize(img, (224, 224))
    
    # Guarda en la carpeta de salida
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, resized_img)

print("Procesamiento completado.")



FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\nicom\\Untitled Folder\\fireDetencion\\Dataset\\train\\no_fire'

In [24]:
import tensorflow as tf

# Rutas de las carpetas
fire_folder = r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset\procesado"
no_fire_folder = r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset\no_fire_procesado"

# Carga las imágenes desde ambas carpetas
dataset = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset",
    labels="inferred",
    label_mode="binary",  # Clasificación binaria: fuego o no fuego
    image_size=(224, 224),  # Tamaño de las imágenes
    batch_size=32
)


Found 10678 files belonging to 2 classes.


In [2]:
# Termina de procesar las imagenes y las normaliza de cara a entrenar el modelo
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size + val_size)

# Normalizar las imágenes (valores entre 0 y 1)
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(normalize)
val_ds = val_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

# Entrenamiento del modelo
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 117s 434ms/step - accuracy: 0.8848 - loss: 1.6223 - val_accuracy: 0.9328 - val_loss: 0.2077
Epoch 2/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 114s 425ms/step - accuracy: 0.9609 - loss: 0.1242 - val_accuracy: 0.9640 - val_loss: 0.1029
Epoch 3/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 115s 429ms/step - accuracy: 0.9900 - loss: 0.0328 - val_accuracy: 0.9773 - val_loss: 0.0715
Epoch 4/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 115s 430ms/step - accuracy: 0.9923 - loss: 0.0267 - val_accuracy: 0.9688 - val_loss: 0.1154
Epoch 5/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 115s 431ms/step - accuracy: 0.9949 - loss: 0.0198 - val_accuracy: 0.9176 - val_loss: 0.2134
Epoch 6/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 122s 457ms/step - accuracy: 0.9930 - loss: 0.0289 - val_accuracy: 0.9763 - val_loss: 0.0664
Epoch 7/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 126s 471ms/step - accuracy: 0.9956 - loss: 0.0144 - val_accuracy: 0.9792 - val_loss: 0.0813
Epoch 8/10
267/267 ━━━━━━━━━━━━━━━━━━━━ 129s 480ms/step - accuracy: 0.9978 -

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Para guardar el modelo
model.save("fire_detection_model.h5")

In [9]:
# Para saber cuales son sus estadísticas finales
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Pérdida en el conjunto de prueba: {test_loss:.4f}")
print(f"Precisión en el conjunto de prueba: {test_accuracy:.4f}")

34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.9601 - loss: 0.1842
Pérdida en el conjunto de prueba: 0.1680
Precisión en el conjunto de prueba: 0.9638


In [10]:
# Pruebas para modificar algunas imagenes de prueba de cara a entrenar mejor el modelo
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configurar aumentos
data_gen = ImageDataGenerator(
    rotation_range=20,          # Rotar imágenes hasta 20 grados
    width_shift_range=0.1,      # Desplazar horizontalmente hasta el 10%
    height_shift_range=0.1,     # Desplazar verticalmente hasta el 10%
    brightness_range=(0.8, 1.2), # Variar brillo entre 80% y 120%
    zoom_range=0.2,             # Acercar o alejar hasta el 20%
    horizontal_flip=True        # Voltear horizontalmente
)

# Generar datos aumentados desde una carpeta de imágenes
train_generator = data_gen.flow_from_directory(
    r"C:\Users\nicom\Untitled Folder\fireDetencion\Dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)


Found 10678 images belonging to 2 classes.


In [15]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Para procesar varias imagenes a la vez
test_folder = r"C:\Users\nicom\Untitled Folder\fireDetencion\test\fire"

for image_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, image_name)
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        print(f"{image_name}: FUEGO detectado con probabilidad de {prediction[0][0] * 100:.2f}%")
    else:
        print(f"{image_name}: SIN FUEGO detectado con probabilidad de {(1 - prediction[0][0]) * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
F_0.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
F_1.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
F_10.jpg: SIN FUEGO detectado con probabilidad de 99.99%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
F_11.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
F_12.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
F_13.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
F_14.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
F_15.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
F_16.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
F_17.jpg: FUEGO detectado con probabilidad de 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
F_18.jpg: FUEGO detectado con probabilida